In [1]:
import keras
from keras.layers import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import multiprocessing as mp
import math

df = pd.read_pickle('df_S300_featurized')
df = df.replace([np.inf, -np.inf, np.nan], 0)
df_X = df.drop('Input Data|S_300_atom',axis=1)
a = {}
for x in df_X.columns:
    if 'ChemEnvSiteFingerprint|GaussianSymmFunc' in x:
        a[x] = 'GaussianSymmFunc|'+x[39:]
df_X = df_X.rename(a,axis=1)

df_Y = df[['Input Data|S_300_atom']]

df_X=(df_X-df_X.min())/(df_X.max()-df_X.min())
#df_Y=(df_Y-df_Y.min())/(df_Y.max()-df_Y.min())
df_X = df_X.replace([np.inf, -np.inf, np.nan], 0)

cross_mi = pd.read_pickle('Features_cross_OLD')
S300_mi = pd.read_pickle('Features_S300')
a = []
for x in cross_mi.index:
    if x not in S300_mi.index:
        a.append(x)
        
cross_mi = cross_mi.drop(a,axis=0).drop(a,axis=1)

def get_features(n_feat):

    first_feature = S300_mi.nlargest(1).index[0]
    feature_set = [first_feature]

    for n in range(n_feat):
        p = 6-2*math.log10(n+1)
        c = 0.0001*(2**(n/10))
        if c > 100000:
            c=100000
        if p < 0.1:
            p=0.1
            
        #0: p = 6-2*math.log10(n+1)
        #0: c = 0.0001*(2**(n/10))
        
        #1: p = 6-2*math.log10(n+1)
        #1: c = 0.0001*(4**(n/10))
        
        # 3
        #p = 1.5
        #c = 0.000001*n**3
        
        #4
        #p = 1.5
        #c = 0.00001*n**3
        
        print((n,p,c))
        score = cross_mi.copy()
        score = score.drop(feature_set,axis=0)
        score = score[feature_set]
        
        for i in score.index:
            row = score.loc[i,:]
            score.loc[i,:] = S300_mi[i] /(row**p+c)
            
        next_feature = score.min(axis=1).idxmax(axis=0)
        feature_set.append(next_feature)
        
    #print(feature_set)
    return feature_set

def merge_df(df1,df2):
    for x in df2.columns:
        for y in df2.index:
            df1.loc[x,y] =+ df2.loc[xy]
    return df1
  
def CV_NN(model,df_X,df_Y,cv=5,epochs=200,batch_size=100,plot = False):
  df_X,df_Y = shuffle(df_X,df_Y)
  size = df_X.shape[0]
  mae_score = np.zeros(cv)
  map_score = np.zeros(cv)
  
  weights = model.get_weights()
  
  for i in range(cv):
    fit_params = {
            'x': df_X.values,
            'y': df_Y.values,
            'epochs': epochs+20,
            'batch_size': batch_size,
            'verbose': 0,
            'validation_split' : 1/cv
    }
    
    model.set_weights(weights)
    history = model.fit(**fit_params)
    mae =  history.history['mean_absolute_error']
    val_map = history.history['val_mean_absolute_percentage_error']
    val_mae = history.history['val_mean_absolute_error']   
    score = np.array(val_mae)[-40:].mean()
    score_p = np.array(val_map)[-40:].mean()
    mae_score[i] = score
    map_score[i] = score_p
    temp = df_X.iloc[:size//cv,:]
    idx = temp.index
    df_X = df_X.drop(idx,axis=0)
    df_X = df_X.append(temp)
    temp = df_Y.loc[idx,:]
    df_Y = df_Y.drop(idx,axis=0)
    df_Y = df_Y.append(temp)
  return (mae_score.mean(), map_score.mean())

def gen_model(n_features):
    f_input = Input(shape=(n_features,))
    hidden1 = Dense(int(n_features/2)+16,activation='relu')(f_input)
    #hidden2 = Dense(int(n_features/2)+2,activation='relu')(hidden1)
    out = Dense(1,activation='linear')(hidden1) #output is bounded below
    model = keras.models.Model(f_input,out)
    model.compile(loss = 'mae', metrics=['mae','mean_absolute_percentage_error'],
                  optimizer=keras.optimizers.Adam(0.01))
    return model

Using TensorFlow backend.


In [3]:
features_l = get_features(1100)
features_l

(0, 6.0, 0.0001)
(1, 5.3979400086720375, 0.00010717734625362932)
(2, 5.045757490560675, 0.00011486983549970352)
(3, 4.795880017344075, 0.00012311444133449164)
(4, 4.6020599913279625, 0.00013195079107728943)
(5, 4.443697499232712, 0.0001414213562373095)
(6, 4.309803919971486, 0.0001515716566510398)
(7, 4.1938200260161125, 0.0001624504792712471)
(8, 4.0915149811213505, 0.00017411011265922483)
(9, 4.0, 0.0001866065983073615)
(10, 3.9172146296835497, 0.0002)
(11, 3.8416375079047502, 0.00021435469250725864)
(12, 3.7721132953863266, 0.000229739670999407)
(13, 3.707743928643524, 0.0002462288826689833)
(14, 3.6478174818886373, 0.00026390158215457886)
(15, 3.5917600346881504, 0.000282842712474619)
(16, 3.5391021572434522, 0.00030314331330207967)
(17, 3.489454989793388, 0.0003249009585424942)
(18, 3.4424927980943423, 0.00034822022531844966)
(19, 3.3979400086720375, 0.000373213196614723)
(20, 3.3555614105321614, 0.0004)
(21, 3.3151546383555877, 0.0004287093850145173)
(22, 3.2765443279648143, 0.00

(184, 1.4656565431939725, 34.590108176164925)
(185, 1.460974111564167, 37.072760009473264)
(186, 1.456316786927002, 39.73360036113022)
(187, 1.45168430147264, 42.58541843808178)
(188, 1.4470763916535114, 45.641921372939855)
(189, 1.4424927980943423, 48.91780010668492)
(190, 1.4379332655045447, 52.4288)
(191, 1.4333975425929006, 56.191796512622865)
(192, 1.4288853819844523, 60.224876314468524)
(193, 1.424396540139548, 64.54742421837798)
(194, 1.4199307772749643, 69.18021635232985)
(195, 1.4154878572870482, 74.14552001894653)
(196, 1.4110675476768142, 79.46720072226044)
(197, 1.4066696194769381, 85.17083687616356)
(198, 1.4022938471805864, 91.28384274587971)
(199, 1.3979400086720375, 97.83560021336984)
(200, 1.393607885159022, 104.8576)
(201, 1.3892972611067522, 112.38359302524573)
(202, 1.385007924173574, 120.44975262893705)
(203, 1.3807396651482025, 129.09484843675597)
(204, 1.3764922778884916, 138.3604327046597)
(205, 1.3722655592616935, 148.29104003789305)
(206, 1.3680593090861644, 1

(391, 0.8134278659590857, 100000)
(392, 0.8112148992491468, 100000)
(393, 0.8090075563488517, 100000)
(394, 0.8068058087470797, 100000)
(395, 0.8046096281489756, 100000)
(396, 0.8024189864737696, 100000)
(397, 0.8002338558526247, 100000)
(398, 0.7980542086265032, 100000)
(399, 0.795880017344075, 100000)
(400, 0.7937112547596357, 100000)
(401, 0.7915478938310603, 100000)
(402, 0.7893899077177808, 100000)
(403, 0.7872372697787897, 100000)
(404, 0.7850899535706626, 100000)
(405, 0.7829479328456115, 100000)
(406, 0.7808111815495602, 100000)
(407, 0.77867967382024, 100000)
(408, 0.7765533839853163, 100000)
(409, 0.7744322865605291, 100000)
(410, 0.7723163562478614, 100000)
(411, 0.770205567933731, 100000)
(412, 0.7680998966871977, 100000)
(413, 0.7659993177582018, 100000)
(414, 0.763903806575815, 100000)
(415, 0.7618133387465145, 100000)
(416, 0.7597278900524849, 100000)
(417, 0.7576474364499299, 100000)
(418, 0.7555719540674097, 100000)
(419, 0.7535014192041993, 100000)
(420, 0.75143580832

(632, 0.39719257996528956, 100000)
(633, 0.3958214842365342, 100000)
(634, 0.3944525494160489, 100000)
(635, 0.39308576870317236, 100000)
(636, 0.3917211353292993, 100000)
(637, 0.3903586425576755, 100000)
(638, 0.38899828368319955, 100000)
(639, 0.38764005203222585, 100000)
(640, 0.3862839409623655, 100000)
(641, 0.38492994386229373, 100000)
(642, 0.3835780541515561, 100000)
(643, 0.3822282652803759, 100000)
(644, 0.3808805707294649, 100000)
(645, 0.37953496400983155, 100000)
(646, 0.37819143866259886, 100000)
(647, 0.37684998825881344, 100000)
(648, 0.3755106063992617, 100000)
(649, 0.3741732867142886, 100000)
(650, 0.37283802286361567, 100000)
(651, 0.37150480853615964, 100000)
(652, 0.3701736374498523, 100000)
(653, 0.36884450335146557, 100000)
(654, 0.3675174000164336, 100000)
(655, 0.36619232124867906, 100000)
(656, 0.3648692608804387, 100000)
(657, 0.3635482127720886, 100000)
(658, 0.3622291708119807, 100000)
(659, 0.36091212891626245, 100000)
(660, 0.3595970810287197, 100000)
(

(869, 0.12096149476276263, 100000)
(870, 0.11996368998467322, 100000)
(871, 0.11896703013486576, 100000)
(872, 0.11797151258886096, 100000)
(873, 0.11697713473119364, 100000)
(874, 0.11598389395537367, 100000)
(875, 0.11499178766383888, 100000)
(876, 0.11400081326791867, 100000)
(877, 0.11301096818779488, 100000)
(878, 0.11202224985245657, 100000)
(879, 0.11103465569966264, 100000)
(880, 0.11004818317590459, 100000)
(881, 0.10906282973636028, 100000)
(882, 0.10807859284486288, 100000)
(883, 0.10709546997385377, 100000)
(884, 0.10611345860434884, 100000)
(885, 0.10513255622589845, 100000)
(886, 0.10415276033654752, 100000)
(887, 0.1031740684427982, 100000)
(888, 0.10219647805957255, 100000)
(889, 0.1012199867101744, 100000)
(890, 0.10024459192625024, 100000)
(891, 0.1, 100000)
(892, 0.1, 100000)
(893, 0.1, 100000)
(894, 0.1, 100000)
(895, 0.1, 100000)
(896, 0.1, 100000)
(897, 0.1, 100000)
(898, 0.1, 100000)
(899, 0.1, 100000)
(900, 0.1, 100000)
(901, 0.1, 100000)
(902, 0.1, 100000)
(903

['AverageBondLength|mean Average bond length',
 'ElementProperty|MagpieData mean Row',
 'GeneralizedRDF|mean Gaussian center=3.0 width=1.0',
 'AtomicOrbitals|HOMO_element',
 'GlobalSymmetryFeatures|crystal_system_int',
 'VoronoiFingerprint|mean Voro_vol_minimum',
 'OPSiteFingerprint|mean square co-planar CN_4',
 'ChemEnvSiteFingerprint|mean A:2',
 'BondOrientationParameter|mean BOOP Q l=7',
 'VoronoiFingerprint|std_dev Voro_vol_sum',
 'OPSiteFingerprint|std_dev q4 CN_10',
 'CrystalNNFingerprint|mean trigonal non-coplanar CN_3',
 'OPSiteFingerprint|std_dev water-like CN_2',
 'CrystalNNFingerprint|std_dev wt CN_4',
 'RadialDistributionFunction|radial distribution function|d_3.60',
 'CrystalNNFingerprint|mean q6 CN_12',
 'OPSiteFingerprint|std_dev sgl_bd CN_1',
 'AGNIFingerPrint|std_dev AGNI dir=z eta=1.04e+01',
 'VoronoiFingerprint|std_dev Voro_dist_maximum',
 'RadialDistributionFunction|radial distribution function|d_2.60',
 'CrystalNNFingerprint|std_dev wt CN_2',
 'CoulombMatrix|coulom

In [4]:
na = [2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 18, 20, 24, 28, 32, 35, 40, 45, 50, 55, 60,
      70, 80, 90, 100, 150, 200, 250, 300, 400, 500, 500,600,700]

na = [800, 900, 1100]

res = []
for n in na:
    mod = gen_model(n)
    x = df_X[features_l[:n]]
    (mae_r,_) = CV_NN(mod,x,df_Y,cv=5,epochs=200,batch_size=100,plot = False)
    res.append(mae_r)
    print(res)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
[2.3749221179667246]
[2.3749221179667246, 2.5221969049426924]
[2.3749221179667246, 2.5221969049426924, 2.5473355595941043]


In [5]:
np.save('dyn_feat_1B',np.array(res))

In [24]:
a

[8.788048479145312,
 8.786278194347059,
 8.83567644321296,
 6.55866032057498,
 6.594281215629424,
 5.9731121386964645,
 6.093298528213578,
 6.013114554470323,
 6.0406105584983365,
 5.925996851854057,
 5.688384171164179,
 5.6050610464069255,
 5.468216931896516,
 5.556654090144069,
 5.378115017983808,
 5.178748119212537,
 5.065057772158619,
 4.947030197166535,
 4.73678596504242,
 4.714628536705991,
 4.61638557714152,
 4.5689975810912715,
 4.413138726542752,
 4.501922417299815,
 4.052189131836336,
 3.9376204579805276,
 3.858442590251026,
 2.8134004539203454,
 2.644128542540064,
 2.506503551317506,
 2.639696221002134,
 2.5707246893428892,
 2.405805012990672,
 2.476441978939087,
 2.4130941697057473,
 2.3717382238859153,
 2.3749221179667246,
 2.5221969049426924,
 2.5473355595941043]

In [2]:
get_features(10)

(0, 6.0, 0.0001)
(1, 5.3979400086720375, 0.00010717734625362932)
(2, 5.045757490560675, 0.00011486983549970352)
(3, 4.795880017344075, 0.00012311444133449164)
(4, 4.6020599913279625, 0.00013195079107728943)
(5, 4.443697499232712, 0.0001414213562373095)
(6, 4.309803919971486, 0.0001515716566510398)
(7, 4.1938200260161125, 0.0001624504792712471)
(8, 4.0915149811213505, 0.00017411011265922483)
(9, 4.0, 0.0001866065983073615)


['AverageBondLength|mean Average bond length',
 'ElementProperty|MagpieData mean Row',
 'GeneralizedRDF|mean Gaussian center=3.0 width=1.0',
 'AtomicOrbitals|HOMO_element',
 'GlobalSymmetryFeatures|crystal_system_int',
 'VoronoiFingerprint|mean Voro_vol_minimum',
 'OPSiteFingerprint|mean square co-planar CN_4',
 'ChemEnvSiteFingerprint|mean A:2',
 'BondOrientationParameter|mean BOOP Q l=7',
 'VoronoiFingerprint|std_dev Voro_vol_sum',
 'OPSiteFingerprint|std_dev q4 CN_10']